In [ ]:
import numpy as np
import pandas as pd
from math import sqrt

In [ ]:
week1=pd.read_csv("nfl-big-data-bowl-2024/tracking_week_1.csv")
week2=pd.read_csv("nfl-big-data-bowl-2024/tracking_week_2.csv")
week3=pd.read_csv("nfl-big-data-bowl-2024/tracking_week_3.csv")
week4=pd.read_csv("nfl-big-data-bowl-2024/tracking_week_4.csv")
week5=pd.read_csv("nfl-big-data-bowl-2024/tracking_week_5.csv")
week6=pd.read_csv("nfl-big-data-bowl-2024/tracking_week_6.csv")
week7=pd.read_csv("nfl-big-data-bowl-2024/tracking_week_7.csv")
week8=pd.read_csv("nfl-big-data-bowl-2024/tracking_week_8.csv")
week9=pd.read_csv("nfl-big-data-bowl-2024/tracking_week_9.csv")
plays=pd.read_csv("nfl-big-data-bowl-2024/plays.csv")
games=pd.read_csv("nfl-big-data-bowl-2024/games.csv")
players=pd.read_csv("nfl-big-data-bowl-2024/players.csv")
tackles=pd.read_csv("nfl-big-data-bowl-2024/tackles.csv")

In [ ]:
all_plays=pd.concat([week1, week2, week3, week4, week5, week6, week7, week8, week9],axis=0)
plays["uniqueid"]=plays["gameId"]*plays["playId"]
all_plays["uniqueid"]=all_plays["gameId"]*all_plays["playId"]
plays_final=pd.merge(plays,games, on="gameId",how="right")
plays_final=plays_final[plays_final["playNullifiedByPenalty"]=="N"]
plays_final.drop(["gameId","playId","ballCarrierDisplayName","playDescription","prePenaltyPlayResult","playNullifiedByPenalty",
                 "foulName1","foulName2","penaltyYards","foulNFLId1","foulNFLId2","season","gameDate","gameTimeEastern"],axis=1,inplace=True)
def assign_offense_defense(row):
    if row['possessionTeam'] == row['homeTeamAbbr']:
        row['offenseScore'] = row['preSnapHomeScore']
        row['defenseScore'] = row['preSnapVisitorScore']
        row['offenseWinProbability'] = row['preSnapHomeTeamWinProbability']
        row['offenseWPAdded']=row["homeTeamWinProbabilityAdded"]
        row['defenseWinProbability'] = row['preSnapVisitorTeamWinProbability']
        row['defenseWPAdded']=row["visitorTeamWinProbilityAdded"]
    else:
        row['offenseScore'] = row['preSnapVisitorScore']
        row['defenseScore'] = row['preSnapHomeScore']
        row['offenseWinProbability'] = row['preSnapVisitorTeamWinProbability']
        row['offenseWPAdded']=row["visitorTeamWinProbilityAdded"]
        row['defenseWinProbability'] = row['preSnapHomeTeamWinProbability']
        row['defenseWPAdded']=row["homeTeamWinProbabilityAdded"]
    return row

plays_final = plays_final.apply(assign_offense_defense, axis=1)
plays_final=plays_final.drop(["preSnapHomeScore","preSnapVisitorScore","preSnapHomeTeamWinProbability","preSnapVisitorTeamWinProbability",
                             "homeFinalScore","visitorFinalScore","homeTeamWinProbabilityAdded","visitorTeamWinProbilityAdded"], axis=1)
all_plays=pd.merge(all_plays, games, on="gameId",how="right")
all_plays.drop(["gameDate","gameTimeEastern","homeTeamAbbr","visitorTeamAbbr","homeFinalScore","visitorFinalScore"],axis=1, inplace=True)
all_id=plays_final.uniqueid
final_data=pd.merge(all_plays, plays_final, on="uniqueid",how="right")
final_data.drop(["gameId","playId","time","jerseyNumber","season","yardlineSide",
                     "yardlineNumber","passResult","passLength","week_y","homeTeamAbbr","visitorTeamAbbr"], axis=1, inplace=True)

In [ ]:
columns_to_check = ['x', 'y', 's', 'a', 'dis']
rows_with_all_nas = final_data[columns_to_check].isna().all(axis=1)
final_data = final_data[~rows_with_all_nas]

In [ ]:
safefumble_ids = final_data[(final_data['event'] == 'fumble') | (final_data['event'] == 'safety')]['uniqueid'].unique()
nfinal_data = final_data[~final_data['uniqueid'].isin(safefumble_ids)]
for index, row in nfinal_data.iterrows():
    if row['playDirection']=="left":
        nfinal_data.loc[index,"x"]=120-row["x"]
        nfinal_data.loc[index,"o"]=360-row["o"]
        nfinal_data.loc[index,"dir"]=360-row["dir"]
        nfinal_data.loc[index,"absoluteYardlineNumber"]=120-row["absoluteYardlineNumber"]
nfinal_data = nfinal_data.copy()

# optional: ensure playResult is numeric
nfinal_data["playResult"] = pd.to_numeric(nfinal_data["playResult"], errors="coerce").fillna(0)

# assign using .loc (all rows)
nfinal_data.loc[:, "AbsolutePostPlay"] = (
    nfinal_data["absoluteYardlineNumber"] + nfinal_data["playResult"]
)
nfinal_data=nfinal_data[nfinal_data["uniqueid"]!=4753982039408]
nfinal_data=nfinal_data[nfinal_data["uniqueid"]!=3912749001045]
nfinal_data=nfinal_data[nfinal_data["uniqueid"]!=4147333259153]

In [ ]:
filtered_groups = nfinal_data.groupby('uniqueid').filter(lambda x: ~((x['event'] == 'handoff') | (x['event'] == 'pass_outcome_caught') | (x["event"]=="run")).any())
unique_ids_without_events = filtered_groups['uniqueid'].unique()
nfinal_data=nfinal_data[~nfinal_data['uniqueid'].isin(unique_ids_without_events)]
relevant_events = nfinal_data[nfinal_data['event'].isin(['handoff', 'run', 'pass_outcome_caught'])]
event_counts = relevant_events.groupby('uniqueid')['event'].nunique()
plays_with_multiple_events = event_counts[event_counts > 1].index.tolist()
nfinal_data=nfinal_data[~nfinal_data['uniqueid'].isin(plays_with_multiple_events)]

In [ ]:
ref_frame_ids = nfinal_data[nfinal_data['event'].isin(['handoff', 'run', 'pass_outcome_caught'])].groupby('uniqueid')['frameId'].min()
nfinal_data = nfinal_data.merge(ref_frame_ids.rename('ref_frameId'), on='uniqueid', how='left')
nfinal_data = nfinal_data[nfinal_data['frameId'] >= nfinal_data['ref_frameId']]
nfinal_data.drop(["ref_frameId"],axis=1, inplace=True)
def calculate_distance(x1, y1, x2, y2):
    return sqrt((x1 - x2)**2 + (y1 - y2)**2)
nfinal_data['tackleOpportunity'] = False
row_counter=0
for unique_id in nfinal_data['uniqueid'].unique():
    play_data = nfinal_data[nfinal_data['uniqueid'] == unique_id]
    possession_team = play_data['possessionTeam'].iloc[0]
    defensive_team = play_data['defensiveTeam'].iloc[0]
    for frame_id in play_data['frameId'].unique():
        frame_data = play_data[play_data['frameId'] == frame_id]
        ball_carrier_row = frame_data[frame_data['nflId'] == frame_data['ballCarrierId'].iloc[0]]
        if not ball_carrier_row.empty:
            ball_carrier_x = ball_carrier_row['x'].iloc[0] 
            ball_carrier_y = ball_carrier_row['y'].iloc[0]
            defenders = frame_data[frame_data['club'] == defensive_team]
            for _, defender in defenders.iterrows():
                row_counter += 1  
                if(row_counter % 10000==0):
                    print(f"Processing row number: {row_counter}")
                distance = calculate_distance(ball_carrier_x, ball_carrier_y, defender['x'], defender['y'])
                if distance <= 2.5:
                    nfinal_data.loc[(nfinal_data['uniqueid'] == unique_id) & (nfinal_data['frameId'] == frame_id) &
                                       (nfinal_data['nflId']== defender['nflId']), 'tackleOpportunity'] = True

In [ ]:
nfinal_data['tackleOpportunity'] = nfinal_data['tackleOpportunity'].astype(int)
defenders_df = nfinal_data[nfinal_data['club'] == nfinal_data['defensiveTeam']]
defender_tackle_opportunities = defenders_df.groupby(['frameId', 'uniqueid']).agg({'tackleOpportunity': 'sum'}).rename(columns={'tackleOpportunity': 'defenderTackleOpportunities'})
new_cmp = nfinal_data.merge(defender_tackle_opportunities, on=['frameId', 'uniqueid'], how='left')
solo_tackle=new_cmp[new_cmp["defenderTackleOpportunities"]==1]
def identify_tackler(sub_df):
    tackler = sub_df[sub_df['tackleOpportunity'] == 1]['nflId'].iloc[0]
    return pd.Series({'tacklerNflId': tackler})
tackler_df = solo_tackle.groupby(['uniqueid', 'frameId']).apply(identify_tackler).reset_index()
solo_tackle = pd.merge(solo_tackle, tackler_df, on=['uniqueid', 'frameId'])


In [ ]:
tackles["uniqueid"]=tackles["gameId"]*tackles["playId"]
tackles=tackles[(tackles["tackle"]==1) | (tackles["assist"]==1)]
index_to_drop = tackles[tackles['uniqueid'] == 3799509190053].index
tackles.drop(index_to_drop, inplace=True)
combos=tackles[tackles["assist"]==1][["uniqueid","nflId"]].drop_duplicates()
tackles=tackles[tackles["tackle"]==1]
tackles.drop(["tackle","gameId","playId","assist","forcedFumble","pff_missedTackle"],axis=1, inplace=True)
solo_tackle = pd.merge(solo_tackle, tackles, on='uniqueid', how='left')
solo_tackle['nflId_y'].fillna(0, inplace=True)
solo_tackle['tackleMade'] = (solo_tackle['nflId_y'] == solo_tackle['tacklerNflId']).astype(int)
solo_tackle.drop(["nflId_y"],axis=1,inplace=True)
solo_tackle=solo_tackle.sort_values(by=["uniqueid","frameId"])
solo_tackle.drop(["defenderTackleOpportunities"],axis=1,inplace=True)

In [ ]:
trial = solo_tackle
ntrial = trial.loc[trial["tackleOpportunity"] == 1].copy()   # <- make an explicit copy
ntrial.reset_index(drop=True, inplace=True)
ntrial.loc[:, 'startTackleOpportunity'] = 0                  # <- use .loc for assignment
ntrial.at[0, 'startTackleOpportunity'] = 1
for i in range(1, len(ntrial)):
    if (ntrial.at[i, 'nflId_x'] == ntrial.at[i - 1, 'nflId_x']) and \
       (ntrial.at[i, 'uniqueid'] == ntrial.at[i - 1, 'uniqueid']) and \
       (ntrial.at[i, 'frameId'] == ntrial.at[i - 1, 'frameId'] + 1):
        ntrial.at[i, 'startTackleOpportunity'] = 0
    else:
        ntrial.at[i, 'startTackleOpportunity'] = 1

ntrial_filtered = ntrial[ntrial['startTackleOpportunity'] == 1]
indices = ntrial_filtered.groupby(['nflId_x', 'uniqueid'])['frameId'].idxmax()
ntrial_filtered = ntrial_filtered.loc[indices]
legal = ntrial_filtered[["frameId", "uniqueid"]]
solo_tackle_final = solo_tackle.merge(legal, on=['frameId', 'uniqueid'], how='inner')
merged_data = solo_tackle.merge(ntrial_filtered, on=['frameId', 'uniqueid'])

In [ ]:
solo_tackle_final=solo_tackle_final[solo_tackle_final["displayName"]!="football"]
solo_tackle_final[['x', 'y']] = solo_tackle_final[['x', 'y']].apply(pd.to_numeric, errors='coerce')
filtered_df = pd.DataFrame()
grouped = solo_tackle_final.groupby(['frameId', 'uniqueid'])
for (frame, unique_id), group in grouped:
    group = group.copy()
    ball_carrier = group[group['nflId_x'] == group['ballCarrierId']]
    tackler = group[group['tackleOpportunity'] == 1]
    if not ball_carrier.empty:
        ball_carrier_x, ball_carrier_y = ball_carrier.iloc[0][['x', 'y']]
        group['distance_to_ball_carrier'] = ((group['x'] - ball_carrier_x) ** 2 + (group['y'] - ball_carrier_y) ** 2) ** 0.5
        distances = group[~group['nflId_x'].isin(ball_carrier['nflId_x']) & ~group['nflId_x'].isin(tackler['nflId_x'])]
        closest_players = distances.nsmallest(3, 'distance_to_ball_carrier')
        frame_filtered = pd.concat([ball_carrier, tackler, closest_players])
        filtered_df = pd.concat([filtered_df, frame_filtered])
filtered_df.reset_index(drop=True, inplace=True)
data_final=filtered_df
data_final.drop(["playDirection","defenseWinProbability","tackleOpportunity"],axis=1, inplace=True)

In [ ]:
pd.set_option("display.max_columns", None)
data_final.head(10)

In [ ]:
def convert_to_seconds(time_str):
    if pd.isna(time_str): 
        return None
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds
data_final['gameClock'] = data_final['gameClock'].apply(convert_to_seconds)
data_final = data_final.merge(players[['nflId', 'height', 'weight']], left_on='ballCarrierId', right_on='nflId', how='left')
data_final.rename(columns={'height': 'ballCarrierHeight', 'weight': 'ballCarrierWeight'}, inplace=True)
data_final.drop(columns='nflId', inplace=True)
data_final = data_final.merge(players[['nflId', 'height', 'weight']], left_on='tacklerNflId', right_on='nflId', how='left')
data_final.rename(columns={'height': 'tacklerHeight', 'weight': 'tacklerWeight'}, inplace=True)
data_final.drop(columns='nflId', inplace=True)
def convert_height_to_inches(height):
    feet, inches = height.split('-')
    return int(feet) * 12 + int(inches)
data_final['tacklerHeight'] = data_final['tacklerHeight'].apply(convert_height_to_inches)
data_final['ballCarrierHeight'] = data_final['ballCarrierHeight'].apply(convert_height_to_inches)

In [ ]:
data_final['side'] = data_final.apply(lambda row: 'offense' if row['club'] == row['possessionTeam'] else 'defense', axis=1)
data_final.drop(["displayName","nflId_x","club","event","possessionTeam","defensiveTeam","distance_to_ball_carrier"],axis=1, inplace=True)
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
data_final["offenseFormation"]=labelencoder.fit_transform(data_final["offenseFormation"])
data_final["side"]=labelencoder.fit_transform(data_final["side"])
data_final['idx'] = data_final.groupby(['uniqueid','frameId']).cumcount() + 1
pivot_df = data_final.pivot(index=['uniqueid','frameId','week_x','ballCarrierId','absoluteYardlineNumber','AbsolutePostPlay',
                                   'quarter','down','yardsToGo',
                                   'offenseScore','defenseScore','defendersInTheBox','offenseFormation',"expectedPoints",
                                   'passProbability','offenseWinProbability','gameClock',
                                  'tacklerHeight','tacklerWeight','ballCarrierHeight','ballCarrierWeight',
                                  'tacklerNflId','tackleMade'], columns='idx')
pivot_df.columns = [f'{col[0]}{col[1]}' for col in pivot_df.columns]
pivot_df.reset_index(inplace=True)
combos.rename(columns={"nflId":"tacklerNflId","uniqueid":"uniqueid"},inplace=True)
merged=pd.merge(pivot_df, combos, how="left",indicator=True,on=["tacklerNflId","uniqueid"])
pivot_dfnew=merged[merged["_merge"]=="left_only"]
pivot_dfnew.drop(columns=["_merge"],inplace=True)

train=pivot_dfnew[pivot_dfnew["week_x"]>=5]
test=pivot_dfnew[pivot_dfnew["week_x"]<5]

In [ ]:
na_count = train.isna().sum()
pd.set_option('display.max_rows', None)
print(na_count)

In [ ]:
train_id = train["uniqueid"].values
train_frame = train["frameId"].values
train_carrier = train["ballCarrierId"].values
train_tackler = train["tacklerNflId"].values

test_id = test["uniqueid"].values
test_frame = test["frameId"].values
test_carrier = test["ballCarrierId"].values
test_tackler = test["tacklerNflId"].values

train = train.copy()
test  = test.copy()

train.drop(["uniqueid","frameId","ballCarrierId","tacklerNflId","AbsolutePostPlay"], axis=1, inplace=True)
test.drop(["uniqueid","frameId","ballCarrierId","tacklerNflId","AbsolutePostPlay"], axis=1, inplace=True)


In [ ]:
y = train["tackleMade"].values
X = train.drop(["tackleMade"], axis=1).values

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
# FIX: use scipy.stats distributions (not numpy/random scalars)
from scipy.stats import randint as sp_randint, uniform as sp_uniform

xgb_model = xgb.XGBClassifier(
    objective='binary:logistic', 
    eval_metric='logloss'
)

param_distributions = {
    'n_estimators':      sp_randint(50, 1001),   # integers [50, 1000]
    'learning_rate':     sp_uniform(0.01, 0.59), # floats in [0.01, 0.60)
    'max_depth':         sp_randint(3, 11),      # integers [3, 10]
    'min_child_weight':  sp_randint(1, 11),      # integers [1, 10]
    'subsample':         sp_uniform(0.5, 0.5),   # [0.5, 1.0)
    'colsample_bytree':  sp_uniform(0.5, 0.5),   # [0.5, 1.0)
    'gamma':             sp_uniform(0.1, 0.6),   # [0.1, 0.7)
    'reg_alpha':         sp_randint(1, 8)        # integers [1, 7]
}

random_search_xgb = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_distributions,
    n_iter=400,
    scoring='roc_auc',
    cv=5,
    random_state=39,
    n_jobs=-1,
    verbose=1,
    error_score="raise"
)
random_search_xgb.fit(X, y)
# Print the best score
print("Best Score:", random_search_xgb.best_score_)

# Print the best parameters
print("Best Parameters:", random_search_xgb.best_params_)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
xgb_model = xgb.XGBClassifier(
    n_estimators=965,
    learning_rate=0.021,
    max_depth=9,
    min_child_weight=2,
    subsample=0.869,
    colsample_bytree=0.909,
    gamma=0.239,
    reg_alpha=1,
    objective='binary:logistic',
    eval_metric='logloss'
)
%time scores = cross_val_score(xgb_model, X, y, cv=kf, scoring='roc_auc')
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')


In [ ]:
mask = ~np.isnan(X).any(axis=1)

X = X[mask]
y = y[mask]

In [ ]:

from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

param_distributions = {
    'n_estimators': randint(50, 1001),
    'max_depth': randint(10, 100),
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 11),
    'bootstrap': [True, False],
}
random_search_rf = RandomizedSearchCV(
    estimator=rf_model, 
    param_distributions=param_distributions,
    n_iter=50, 
    scoring='roc_auc',
    cv=5, 
    random_state=30, 
    n_jobs=-1, 
    verbose=1
)
%time random_search_rf.fit(X,y)
# Print the best score
print("Best Score:", random_search_rf.best_score_)

# Print the best parameters
print("Best Parameters:", random_search_rf.best_params_)

np.random.seed(42)
kf=KFold(n_splits=5, shuffle=True, random_state=42)
rf_model = RandomForestClassifier(bootstrap=False, min_samples_leaf=4, min_samples_split=9, n_estimators=937,random_state=42, max_depth=27,
                                 )
%time scores=cross_val_score(rf_model, X, y, cv=kf, scoring='roc_auc')
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')


In [ ]:
from catboost import CatBoostClassifier
cat_model = CatBoostClassifier(random_state=42, verbose=0)

param_distributions = {
    'learning_rate': uniform(0, 1),
    'depth': randint(4, 10),
    'l2_leaf_reg': uniform(0, 10),
    'n_estimators': randint(100, 1001),
    'border_count': randint(32, 255),
    'min_data_in_leaf': randint(1, 50),
    'bagging_temperature': uniform(0, 1),
    'random_strength': randint(1, 20),
}
random_search_cat = RandomizedSearchCV(
    estimator=cat_model, 
    param_distributions=param_distributions,
    n_iter=200, 
    scoring='roc_auc',  # Change scoring if needed
    cv=5, 
    random_state=38, 
    n_jobs=-1, 
    verbose=1,
    error_score="raise"
)
%time random_search_cat.fit(X, y)

# Print the best score
print("Best Score:", random_search_cat.best_score_)

# Print the best parameters
print("Best Parameters:", random_search_cat.best_params_)

np.random.seed(42)
cat_model = CatBoostClassifier(iterations=612,
                          learning_rate=0.030,
                          depth=6,
                          eval_metric='Logloss',
                          random_seed=35,
                          bagging_temperature=0.743,
                          border_count=99,
                          l2_leaf_reg=9.279,
                          min_data_in_leaf=19,
                          random_strength=11,
                          od_type='Iter',
                          metric_period=50,
                          od_wait=20,verbose=0
                        )
%time scores=cross_val_score(cat_model, X, y, cv=kf, scoring='roc_auc',error_score="raise")
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')


In [ ]:

from sklearn.ensemble import GradientBoostingClassifier
gdbt_model = GradientBoostingClassifier(random_state=42, verbose=0)

param_distributions = {
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 10),
    'min_samples_split':randint(2,20), 
    'n_estimators': randint(100, 1001),
    'min_samples_leaf': randint(1, 20),
}
random_search_gdbt = RandomizedSearchCV(
    estimator=gdbt_model, 
    param_distributions=param_distributions,
    n_iter=25, 
    scoring='roc_auc', 
    cv=5, 
    random_state=34, 
    n_jobs=-1, 
    verbose=1,
    error_score="raise"
)

%time random_search_gdbt.fit(X, y)
# Print the best score
print("Best Score:", random_search_gdbt.best_score_)

# Print the best parameters
print("Best Parameters:", random_search_gdbt.best_params_)

np.random.seed(25)
gdbt_model = GradientBoostingClassifier(learning_rate=0.02, max_depth=8, min_samples_leaf=18, min_samples_split=3, 
                    n_estimators=617, verbose=0)
%time scores=cross_val_score(gdbt_model, X, y, cv=kf, scoring='roc_auc',error_score="raise")
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')


In [ ]:

from sklearn.ensemble import ExtraTreesClassifier
extra_model = ExtraTreesClassifier(random_state=33)

param_distributions={
    "n_estimators":randint(100,1000),
    "min_samples_split":randint(2,20),
    "min_samples_leaf":randint(1,20),
    "min_weight_fraction_leaf":uniform(0,0.5),
    "min_impurity_decrease":uniform(0,1),
}
random_search_ext = RandomizedSearchCV(estimator=extra_model, param_distributions=param_distributions,n_iter=500, 
                           scoring='roc_auc', cv=5, random_state=35, n_jobs=-1, verbose=1,error_score="raise")
%time random_search_ext.fit(X,y)
# Print the best score
print("Best Score:", random_search_ext.best_score_)

# Print the best parameters
print("Best Parameters:", random_search_ext.best_params_)

np.random.seed(34)
ext_model = ExtraTreesClassifier(min_impurity_decrease=0.0174,min_samples_leaf=16,min_samples_split=9, 
                                 min_weight_fraction_leaf=0.1243, n_estimators=272,verbose=0)
%time scores=cross_val_score(ext_model, X, y, cv=kf, scoring='roc_auc',error_score="raise")
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')


In [ ]:
from sklearn.svm import SVC
np.random.seed(34)
svm_model = SVC(kernel="poly",probability=True)
%time scores=cross_val_score(svm_model, X, y, cv=kf, scoring='roc_auc',error_score="raise")
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
np.random.seed(34)
knn_model = KNeighborsClassifier(n_neighbors=70)
%time scores=cross_val_score(knn_model, X, y, cv=kf, scoring='roc_auc',error_score="raise")
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')

In [ ]:
from sklearn.linear_model import LogisticRegression
np.random.seed(40)
log_model = LogisticRegression(max_iter=10000)
%time scores=cross_val_score(log_model, X, y, cv=kf, scoring='roc_auc',error_score="raise")
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')

In [ ]:
from sklearn.ensemble import StackingClassifier
base_models=[
    ('RandomForest',rf_model),("Catboost",cat_model),
    ('GDBT',gdbt_model),('Ett',extra_model),('SVM',svm_model),('KNN',knn_model),('Logistic',log_model)
]
stack_clf = StackingClassifier(
     estimators=base_models,
    final_estimator=xgb_model
)
%time scores=cross_val_score(stack_clf, X, y, cv=kf, scoring='roc_auc',error_score="raise")
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')

In [ ]:

clf1=xgb_model.fit(X,y)
clf2=rf_model.fit(X,y)
clf3=cat_model.fit(X,y)
clf4=gdbt_model.fit(X,y)
clf5=extra_model.fit(X,y)
clf6=svm_model.fit(X,y)
clf7=knn_model.fit(X,y)
clf8=log_model.fit(X,y)
weights=[0.2, 0.15, 0.2, 0.1, 0.05,0.05,0.05, 0.2]

y_test=test["tackleMade"].values
X_test=test.drop(["tackleMade"], axis=1).values
predictions = np.array([clf1.predict_proba(X_test), clf2.predict_proba(X_test), clf3.predict_proba(X_test),
                       clf4.predict_proba(X_test), clf5.predict_proba(X_test),clf6.predict_proba(X_test),
                       clf7.predict_proba(X_test), clf8.predict_proba(X_test)])
weighted_predictions = np.tensordot(predictions, weights, axes=((0),(0)))
final_probabilities = weighted_predictions / weighted_predictions.sum(axis=1, keepdims=True)
prob=final_probabilities[:,1]

# if prob is same length as test
test.loc[:, "tackleProbability"] = prob

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
predictions=np.where(prob>=0.5,1,0)
test = test.copy()  # optional but shuts up SettingWithCopy warnings
test.loc[:, "predictions"] = predictions

roc_auc = roc_auc_score(y_test, predictions)
print(f'ROC AUC Score: {roc_auc}')
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')
f1 = f1_score(y_test, predictions)
print(f'F1 Score: {f1}')
roc_auc = roc_auc_score(y_test, prob)
print(f'ROC AUC Score for Probability: {roc_auc}')

In [ ]:
table={'Model': ["XGBoost","Random Forest","CatBoost","Gradient Boosting Decision Tree","Extra Trees","Support Vector Machine","K-Nearest Neighbor","Logistic Regression","Stacked Classifier","Weighted Classifier(Selected)"],
      'ROC-AUC Score':[0.79,0.79,0.80,0.79,0.73,0.73,0.69,0.77,0.77,0.81],
      'Training Time(Seconds)':[22.7,141,7.89,448,1.43,26.4,0.494,149,3863,756]}
print(pd.DataFrame(table))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_mat = confusion_matrix(y_test, predictions)
confusion_mat


In [ ]:
feature_importances = xgb_model.feature_importances_
train.drop(["tackleMade"],axis=1,inplace=True)
feature_names = train.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)

In [ ]:

min_ids=nfinal_data.groupby('uniqueid')["frameId"].min().reset_index()
new_final=pd.merge(nfinal_data, min_ids, on=["uniqueid","frameId"])
new_final['iscarrier'] = (new_final['nflId'] == new_final['ballCarrierId']).astype(int)
for index, row in new_final.iterrows():
    if(row["club"]==row["possessionTeam"]):
        new_final.loc[index,"side"]="Offense"
    else:
        new_final.loc[index,"side"]="Defense"
    if(row["displayName"]=="football"):
        new_final.loc[index,"side"]="Football"
new_final.drop(["displayName","frameId","club","playDirection","event","possessionTeam","defensiveTeam","AbsolutePostPlay"],axis=1,inplace=True)
plays=plays[plays["playNullifiedByPenalty"]=='N']
plays["uniqueid"]=plays["gameId"]*plays["playId"]
plays=plays[['uniqueid','prePenaltyPlayResult']]
new_final=pd.merge(new_final, plays, on="uniqueid",how="left")
new_final["AbsolutePostPlay"]=new_final["absoluteYardlineNumber"]+new_final["prePenaltyPlayResult"]
new_final.drop(["prePenaltyPlayResult"],axis=1,inplace=True)

In [ ]:

new_final.drop(["nflId","ballCarrierId","tackleOpportunity"],axis=1,inplace=True)
new_final['idx'] = new_final.groupby(['uniqueid']).cumcount() + 1
new_final = new_final[~new_final['uniqueid'].isin([127391783463,3043263972035,6005641766850])]
def calculate_distance(group):
    ball_carrier_x = group.loc[group['iscarrier'] == 1, 'x'].iloc[0]
    ball_carrier_y = group.loc[group['iscarrier'] == 1, 'y'].iloc[0]
    group['distance_to_carrier'] = np.sqrt((group['x'] - ball_carrier_x) ** 2 + (group['y'] - ball_carrier_y) ** 2)
    return group

new_final = new_final.groupby('uniqueid').apply(calculate_distance).reset_index(drop=True)
new_final['side'] = pd.Categorical(new_final['side'], categories=["Offense", "Defense", "Football"], ordered=True)

sorted_final = new_final.sort_values(by=['uniqueid', 'side', 'distance_to_carrier'])
sorted_final['idx'] = sorted_final.groupby(['uniqueid']).cumcount() + 1
sorted_final.drop(["defenseWinProbability","iscarrier","side"],axis=1,inplace=True)
pivot_dataf = sorted_final.pivot(index=['uniqueid','week_x','absoluteYardlineNumber','AbsolutePostPlay',
                                   'quarter','down','yardsToGo','offenseFormation','expectedPoints',
                                   'offenseScore','defenseScore','defendersInTheBox',
                                   'passProbability','offenseWinProbability','gameClock'], columns='idx')
pivot_dataf.columns = [f'{col[0]}{col[1]}' for col in pivot_dataf.columns]
pivot_dataf.reset_index(inplace=True)


In [ ]:
def convert_to_seconds(time_str):
    if pd.isna(time_str): 
        return None
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds
pivot_dataf['gameClock'] = pivot_dataf['gameClock'].apply(convert_to_seconds)
pivot_dataf['TD'] = np.where(pivot_dataf['AbsolutePostPlay'] >= 110, 1, 0)
pivot_dataf.drop(["dir23","o23"],axis=1,inplace=True)
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
pivot_dataf["offenseFormation"]=label_encoder.fit_transform(pivot_dataf["offenseFormation"])

In [ ]:
train2=pivot_dataf[pivot_dataf["week_x"]>=5]
test2=pivot_dataf[pivot_dataf["week_x"]<=4]
y2=train2["TD"].values
X2=train2.drop(["AbsolutePostPlay","TD","uniqueid"],axis=1).values

In [ ]:

xgb_model = xgb.XGBClassifier(
    objective='binary:logistic', 
    eval_metric='logloss'  
)
param_distributions={
    'n_estimators':randint(50,1001),
    'learning_rate':uniform(0.01,0.59),
    'max_depth':randint(3,11),
    'min_child_weight':randint(1,11),
    'subsample':uniform(0.5,0.5),
    'colsample_bytree':uniform(0.5,0.5),
    'gamma':uniform(0.1,0.6),
    'reg_alpha':randint(1,8)
}
random_search_xgb = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_distributions,n_iter=400, 
                           scoring='roc_auc', cv=5, random_state=39, n_jobs=-1, verbose=1,error_score="raise")
random_search_xgb.fit(X2,y2)
# Print the best score
print("Best Score:", random_search_xgb.best_score_)

# Print the best parameters
print("Best Parameters:", random_search_xgb.best_params_)

kf=KFold(n_splits=5, shuffle=True, random_state=42)
xgb_model=xgb.XGBClassifier(
    n_estimators=965,
    learning_rate=0.021,
    max_depth=9,
    min_child_weight=2,
    subsample=0.869,
    colsample_bytree=0.909,
    gamma=0.239,
    reg_alpha=1,
    objective='binary:logistic',
    eval_metric='logloss'
)
%time scores=cross_val_score(xgb_model, X2, y2, cv=kf, scoring='roc_auc')

for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')

In [ ]:
mask = ~np.isnan(X2).any(axis=1)

X2 = X2[mask]
y2 = y2[mask]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
param_distributions = {
    'n_estimators': randint(50, 1001),
    'max_depth': randint(10, 100),
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 11),
    'bootstrap': [True, False],
}
random_search_rf = RandomizedSearchCV(
    estimator=rf_model, 
    param_distributions=param_distributions,
    n_iter=50, 
    scoring='roc_auc',
    cv=5, 
    random_state=46, 
    n_jobs=-1, 
    verbose=1
)
%time random_search_rf.fit(X2,y2)
# Print the best score
print("Best Score:", random_search_rf.best_score_)

# Print the best parameters
print("Best Parameters:", random_search_rf.best_params_)
np.random.seed(42)
kf=KFold(n_splits=5, shuffle=True, random_state=42)
rf_model = RandomForestClassifier(bootstrap=False, min_samples_leaf=3, min_samples_split=9, n_estimators=460,random_state=42, max_depth=10,
                                )

%time scores=cross_val_score(rf_model, X22, y2, cv=kf, scoring='roc_auc')
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')


In [ ]:
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier(random_state=42, verbose=0)
param_distributions = {
    'learning_rate': uniform(0, 1),
    'depth': randint(4, 10),
    'l2_leaf_reg': uniform(0, 10),
    'n_estimators': randint(100, 1001),
    'border_count': randint(32, 255),
    'min_data_in_leaf': randint(1, 50),
    'bagging_temperature': uniform(0, 1),
    'random_strength': randint(1, 20),
}
random_search_cat = RandomizedSearchCV(
    estimator=cat_model, 
    param_distributions=param_distributions,
    n_iter=200, 
    scoring='roc_auc',  # Change scoring if needed
    cv=5, 
    random_state=35, 
    n_jobs=-1, 
    verbose=1,
    error_score="raise"
)
%time random_search_cat.fit(X2, y2)

# Print the best score
print("Best Score:", random_search_cat.best_score_)

# Print the best parameters
print("Best Parameters:", random_search_cat.best_params_)

np.random.seed(42)
cat_model = CatBoostClassifier(iterations=877,
                          learning_rate=0.007,
                          depth=8,
                          eval_metric='Logloss',
                          random_seed=35,
                          bagging_temperature=0.825,
                          border_count=187,
                          l2_leaf_reg=6.951,
                          min_data_in_leaf=31,
                          random_strength=1,
                          od_type='Iter',
                          metric_period=50,
                          od_wait=20,verbose=0
                        )

%time scores=cross_val_score(cat_model, X22, y2, cv=kf, scoring='roc_auc',error_score="raise")
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')


In [ ]:

from sklearn.ensemble import GradientBoostingClassifier

gdbt_model = GradientBoostingClassifier(random_state=42, verbose=0)
param_distributions = {
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 10),
    'min_samples_split':randint(2,20), 
    'n_estimators': randint(100, 1001),
    'min_samples_leaf': randint(1, 20),
}
random_search_gdbt = RandomizedSearchCV(
    estimator=gdbt_model, 
    param_distributions=param_distributions,
    n_iter=20, 
    scoring='roc_auc', 
    cv=5, 
    random_state=34, 
    n_jobs=-1, 
    verbose=1,
    error_score="raise"
)

%time random_search_gdbt.fit(X2, y2)
# Print the best score
print("Best Score:", random_search_gdbt.best_score_)

# Print the best parameters
print("Best Parameters:", random_search_gdbt.best_params_)

np.random.seed(25)
gdbt_model = GradientBoostingClassifier(learning_rate=0.01, max_depth=5, min_samples_leaf=15, min_samples_split=8, 
                    n_estimators=473, verbose=0)

%time scores=cross_val_score(gdbt_model, X22, y2, cv=kf, scoring='roc_auc',error_score="raise")
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')


In [ ]:

from sklearn.ensemble import ExtraTreesClassifier

extra_model = ExtraTreesClassifier(random_state=33)
param_distributions={
    "n_estimators":randint(100,1000),
    "min_samples_split":randint(2,20),
    "min_samples_leaf":randint(1,20),
    "min_weight_fraction_leaf":uniform(0,0.5),
    "min_impurity_decrease":uniform(0,1),
}
random_search_ext = RandomizedSearchCV(estimator=extra_model, param_distributions=param_distributions,n_iter=100, 
                           scoring='roc_auc', cv=5, random_state=33, n_jobs=-1, verbose=1,error_score="raise")
%time random_search_ext.fit(X2,y2)
# Print the best score
print("Best Score:", random_search_ext.best_score_)

# Print the best parameters
print("Best Parameters:", random_search_ext.best_params_)

np.random.seed(34)
ext_model = ExtraTreesClassifier(min_impurity_decrease=0.0102,min_samples_leaf=1,min_samples_split=10, 
                                 min_weight_fraction_leaf=0.1086, n_estimators=960,verbose=0)

%time scores=cross_val_score(ext_model, X22, y2, cv=kf, scoring='roc_auc',error_score="raise")
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')


In [ ]:
from sklearn.svm import SVC
np.random.seed(34)
svm_model = SVC(kernel="poly",probability=True)

%time scores=cross_val_score(svm_model, X22, y2, cv=kf, scoring='roc_auc',error_score="raise")
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
np.random.seed(34)
knn_model = KNeighborsClassifier(n_neighbors=70)

%time scores=cross_val_score(knn_model, X22, y2, cv=kf, scoring='roc_auc',error_score="raise")
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')


In [ ]:
from sklearn.linear_model import LogisticRegression
np.random.seed(40)
log_model = LogisticRegression(max_iter=10000)

%time scores=cross_val_score(log_model, X22, y2, cv=kf, scoring='roc_auc',error_score="raise")
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')'

In [ ]:
from sklearn.ensemble import StackingClassifier
base_models=[
    ('RandomForest',rf_model),("Catboost",cat_model),
    ('GDBT',gdbt_model),('Ett',extra_model),('SVM',svm_model),('KNN',knn_model),('Logistic',log_model)
]
stack_clf = StackingClassifier(
     estimators=base_models,
    final_estimator=xgb_model
)

%time scores=cross_val_score(stack_clf, X2, y2, cv=kf, scoring='roc_auc',error_score="raise")
for i, score in enumerate(scores):
    print(f'Fold {i+1}:  = {score:.2f}')
avg_score = np.mean(scores)
print(f'Average SCORE: {avg_score:.2f}')


In [ ]:

clf11=xgb_model.fit(X2,y2)
clf21=rf_model.fit(X2,y2)
clf31=cat_model.fit(X2,y2)
clf41=gdbt_model.fit(X2,y2)
clf51=extra_model.fit(X2,y2)
clf61=svm_model.fit(X2,y2)
clf71=knn_model.fit(X2,y2)
clf81=log_model.fit(X2,y2)
weights=[0.2, 0.15, 0.15, 0.15, 0.05,0.05,0.05, 0.2]
y_test2=test2["TD"].values
X_test2=test2.drop(["TD","uniqueid","AbsolutePostPlay"], axis=1).values
predictions2 = np.array([clf11.predict_proba(X_test2), clf21.predict_proba(X_test2), clf31.predict_proba(X_test2),
                       clf41.predict_proba(X_test2), clf51.predict_proba(X_test2),clf61.predict_proba(X_test2),
                       clf71.predict_proba(X_test2), clf81.predict_proba(X_test2)])
weighted_predictions2 = np.tensordot(predictions2, weights, axes=((0),(0)))
final_probabilities2 = weighted_predictions2 / weighted_predictions2.sum(axis=1, keepdims=True)

prob2=final_probabilities2[:,1]
predictions2=np.where(prob2>=0.25,1,0)
test2.loc[:, "predictions"] = predictions2

In [ ]:
from sklearn.metrics import f1_score
#test2[test2["predictions"]==1]
confusion_mat = confusion_matrix(y_test2, predictions2)
confusion_mat
#f1_score(y_test2, predictions2)
#roc_auc_score(y_test2, prob22)

In [ ]:
train3=train2[train2["TD"]==0]
y3=train3["AbsolutePostPlay"].values

X3=train3.drop(["AbsolutePostPlay","TD","uniqueid"],axis=1).values
xgb_model = xgb.XGBRegressor(objective='reg:squarederror',eval_metric='rmse')


# Grid Search for xgboost
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

param_distributions={
    'n_estimators':randint(50,1001),
    'learning_rate':uniform(0.01,0.59),
    'max_depth':randint(3,11),
    'min_child_weight':randint(1,11),
    'subsample':uniform(0.5,0.5),
    'colsample_bytree':uniform(0.5,0.5),
    'gamma':uniform(0.1,0.6),
    'reg_alpha':randint(1,8)
}
random_search_xgb = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_distributions,n_iter=150, 
                           scoring='neg_mean_squared_error', cv=5, random_state=25, n_jobs=-1, verbose=1,error_score="raise")

%time random_search_xgb.fit(X3,y3)


In [ ]:
mask = ~np.isnan(X3).any(axis=1)

X3 = X3[mask]
y3 = y3[mask]

In [ ]:
kf=KFold(n_splits=5, shuffle=True, random_state=42)

xgb_model=xgb.XGBRegressor(
    n_estimators=279,
    learning_rate=0.038,
    max_depth=3,
    min_child_weight=9,
    subsample=0.608,
    colsample_bytree=0.571,
    gamma=0.445,
    reg_alpha=4,
    reg_lambda=1,
    objective='reg:squarederror',
    eval_metric='rmse'
)


%time scores=cross_val_score(xgb_model, X3, y3, cv=kf, scoring='neg_mean_squared_error')

rmse = np.sqrt(-scores)

# Display the RMSE for each fold
for i, score in enumerate(rmse):
    print(f'Fold {i+1}: RMSE = {score:.2f}')

# Calculate and display the average RMSE across all folds
avg_rmse = np.mean(rmse)
print(f'Average RMSE: {avg_rmse:.2f}')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(criterion="squared_error",random_state=42,n_jobs=-1)

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

param_distributions={
    'n_estimators':randint(50,1001),
    'max_depth':randint(10,100),
    'min_samples_split':randint(2,11),
    'min_samples_leaf':randint(1,11),
    'bootstrap':[True, False],
}
random_search_rf = RandomizedSearchCV(estimator=rf_model, param_distributions=param_distributions,n_iter=15, 
                           scoring='neg_mean_squared_error', cv=3, random_state=25, n_jobs=-1, verbose=1,error_score="raise")

%time random_search_rf.fit(X3,y3)


In [ ]:
print("Best parameters:",random_search_rf.best_params_)
print("Best score:",-random_search_rf.best_score_)

In [ ]:
#print("Best parameters:",random_search_rf.best_params_)
#print("Best score:",-random_search_rf.best_score_)
# Setup random seed for reproducibility
np.random.seed(42)

# Initialize RandomForestRegressor
rf_regressor = RandomForestRegressor(bootstrap=True, min_samples_leaf=9, min_samples_split=8, n_estimators=549,random_state=42, max_depth=24)

# Perform K-Fold CV and calculate RMSE
%time mse_scores = cross_val_score(rf_regressor, X3, y3, cv=kf, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-mse_scores)

# Display the RMSE for each fold
for i, rmse in enumerate(rmse_scores, 1):
    print(f'Fold {i}: RMSE = {rmse:.2f}')

# Calculate and display the average RMSE
avg_rmse = np.mean(rmse_scores)
print(f'Average RMSE: {avg_rmse:.2f}')

In [ ]:
from sklearn.linear_model import Ridge
ridge_model=Ridge(alpha=1)

%time mse_scores = cross_val_score(ridge_model, X3, y3, cv=kf, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-mse_scores)

# Display the RMSE for each fold
for i, rmse in enumerate(rmse_scores, 1):
    print(f'Fold {i}: RMSE = {rmse:.2f}')

# Calculate and display the average RMSE
avg_rmse = np.mean(rmse_scores)
print(f'Average RMSE: {avg_rmse:.2f}')

In [ ]:
from sklearn.linear_model import Lasso
lasso_model=Lasso(alpha=0.1)

%time mse_scores = cross_val_score(lasso_model, X3, y3, cv=kf, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-mse_scores)

# Display the RMSE for each fold
for i, rmse in enumerate(rmse_scores, 1):
    print(f'Fold {i}: RMSE = {rmse:.2f}')

# Calculate and display the average RMSE
avg_rmse = np.mean(rmse_scores)
print(f'Average RMSE: {avg_rmse:.2f}')

In [ ]:
from catboost import CatBoostRegressor

cat_model = CatBoostRegressor(random_state=42,verbose=0)
from scipy.stats import uniform, randint

param_distributions={
    'learning_rate':uniform(0,1),
    'depth':randint(4,10),
    'l2_leaf_reg':uniform(0,10),
    'n_estimators':randint(100,1001),
    'border_count':randint(32,255),
    'min_data_in_leaf':randint(1,50),
    'bagging_temperature':uniform(0,1),
    'random_strength':randint(1,20),
}
from sklearn.model_selection import RandomizedSearchCV
random_search_cat = RandomizedSearchCV(estimator=cat_model, param_distributions=param_distributions,n_iter=80, 
                           scoring='neg_mean_squared_error', cv=5, random_state=35, n_jobs=-1, verbose=1,error_score="raise")

%time random_search_cat.fit(X2,y2)


In [ ]:
print("Best parameters:",random_search_cat.best_params_)
print("Best score:",-random_search_cat.best_score_)

In [ ]:
cat_model = CatBoostRegressor(iterations=751,
                          learning_rate=0.028,
                          depth=5,
                          eval_metric='RMSE',
                          random_seed=35,
                          bagging_temperature=0.458,
                          border_count=215,
                          l2_leaf_reg=2.4611,
                          min_data_in_leaf=41,
                          random_strength=12,
                          od_type='Iter',
                          metric_period=50,
                          od_wait=20,verbose=0)
%time mse_scores = cross_val_score(cat_model, X3, y3, cv=kf, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-mse_scores)

# Display the RMSE for each fold
for i, rmse in enumerate(rmse_scores, 1):
    print(f'Fold {i}: RMSE = {rmse:.2f}')

# Calculate and display the average RMSE
avg_rmse = np.mean(rmse_scores)
print(f'Average RMSE: {avg_rmse:.2f}')

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
extra_model = ExtraTreesRegressor(random_state=42)

param_distributions={
    "n_estimators":randint(100,1000),
    "min_samples_split":randint(2,20),
    "min_samples_leaf":randint(1,20),
    "min_weight_fraction_leaf":uniform(0,0.5),
    "min_impurity_decrease":uniform(0,1),
}

random_search_ext = RandomizedSearchCV(estimator=extra_model, param_distributions=param_distributions,n_iter=160, 
                           scoring='neg_mean_squared_error', cv=5, random_state=35, n_jobs=-1, verbose=1,error_score="raise")

%time random_search_ext.fit(X2,y2)


In [ ]:

print("Best parameters:",random_search_ext.best_params_)
print("Best score:",-random_search_ext.best_score_)


In [ ]:

extra_model=ExtraTreesRegressor(n_estimators=594, random_state=35,min_impurity_decrease=0.019, min_samples_leaf=5,
                               min_samples_split=6,min_weight_fraction_leaf=0.0057)
%time mse_scores = cross_val_score(extra_model, X3, y3, cv=kf, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-mse_scores)

# Display the RMSE for each fold
for i, rmse in enumerate(rmse_scores, 1):
    print(f'Fold {i}: RMSE = {rmse:.2f}')

# Calculate and display the average RMSE
avg_rmse = np.mean(rmse_scores)
print(f'Average RMSE: {avg_rmse:.2f}')

In [ ]:
from sklearn.linear_model import ElasticNet

elastic_net_model = ElasticNet(alpha=1.0, l1_ratio=0.5)
%time mse_scores = cross_val_score(elastic_net_model, X3, y3, cv=kf, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-mse_scores)

# Display the RMSE for each fold
for i, rmse in enumerate(rmse_scores, 1):
    print(f'Fold {i}: RMSE = {rmse:.2f}')

# Calculate and display the average RMSE
avg_rmse = np.mean(rmse_scores)
print(f'Average RMSE: {avg_rmse:.2f}')

In [ ]:
from sklearn.svm import SVR
svr_model = SVR(C=1.0, epsilon=0.1, kernel='rbf')
%time mse_scores = cross_val_score(svr_model, X3, y3, cv=kf, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-mse_scores)

# Display the RMSE for each fold
for i, rmse in enumerate(rmse_scores, 1):
    print(f'Fold {i}: RMSE = {rmse:.2f}')

# Calculate and display the average RMSE
avg_rmse = np.mean(rmse_scores)
print(f'Average RMSE: {avg_rmse:.2f}')

In [ ]:
from sklearn.ensemble import StackingRegressor
base_learners = [
    ('ridge', ridge_model),
    ('lasso', xgb_model),
    ('rf', rf_regressor),
    ('extra', extra_model),
    ('catboost',cat_model),
    ('elasticnet',elastic_net_model)
]

stack_model=StackingRegressor(estimators=base_learners, 
                             final_estimator=lasso_model)
%time mse_scores = cross_val_score(stack_model, X3, y3, cv=kf, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-mse_scores)

# Display the RMSE for each fold
for i, rmse in enumerate(rmse_scores, 1):
    print(f'Fold {i}: RMSE = {rmse:.2f}')

# Calculate and display the average RMSE
avg_rmse = np.mean(rmse_scores)
print(f'Average RMSE: {avg_rmse:.2f}')

In [ ]:
stack_model.fit(X3,y3)

In [ ]:
stack_model

In [ ]:
test3=test2[test2["TD"]==0]
test3 = test3[train3.columns]  
X_test3=test3.drop(["uniqueid","AbsolutePostPlay","TD"],axis=1).values
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pred3 = stack_model.predict(X_test3)
test3.loc[:, "predictions"] = pred3

test3['residuals'] =  test3['predictions'] - test3['AbsolutePostPlay']

plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
plt.scatter(test3['predictions'], test3['residuals'])
plt.title('Residuals vs. Predictions')
plt.xlabel('Predictions (pred3)')
plt.ylabel('Residuals')
plt.axhline(y=0, color='r', linestyle='-')

plt.subplot(1, 2, 2)
sns.histplot(test3['residuals'], kde=True)
plt.title('Distribution of Residuals')
plt.xlabel('Residuals')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
made_tackle=data_final[(data_final["tackleMade"]==1) & (data_final["week_x"]<=4)]
nfinal_data.drop(["AbsolutePostPlay"], axis=1, inplace=True)
nfinal_data=pd.merge(nfinal_data,plays,on="uniqueid",how="left")
nfinal_data["AbsolutePostplay"]=nfinal_data["absoluteYardlineNumber"]+nfinal_data["prePenaltyPlayResult"]
nfinal_data.drop(["prePenaltyPlayResult"],axis=1,inplace=True)
combos=made_tackle[['uniqueid','frameId','tacklerNflId']].drop_duplicates()
nfinal_data=nfinal_data.merge(combos, on=["uniqueid","frameId"])

In [ ]:
for index, row in nfinal_data.iterrows():
    if(row["club"]==row["possessionTeam"]):
        nfinal_data.loc[index,"side"]="Offense"
    else:
        nfinal_data.loc[index,"side"]="Defense"
    if(row["displayName"]=="football"):
        nfinal_data.loc[index,"side"]="Football"
nfinal_data['iscarrier'] = (nfinal_data['nflId'] == nfinal_data['ballCarrierId']).astype(int)
def calculate_distance(group):
    ball_carrier_x = group.loc[group['iscarrier'] == 1, 'x'].iloc[0]
    ball_carrier_y = group.loc[group['iscarrier'] == 1, 'y'].iloc[0]
    group['distance_to_carrier'] = np.sqrt((group['x'] - ball_carrier_x) ** 2 + (group['y'] - ball_carrier_y) ** 2)
    return group
nfinal_data = nfinal_data.groupby('uniqueid').apply(calculate_distance).reset_index(drop=True)
nfinal_data['side'] = pd.Categorical(nfinal_data['side'], categories=["Offense", "Defense", "Football"], ordered=True)
sorted_final2 = nfinal_data.sort_values(by=['uniqueid', 'side', 'distance_to_carrier'])

In [ ]:
def update_coordinates(group):
    row_22_x = group.iloc[21]['x']
    row_22_y = group.iloc[21]['y']
    group.loc[group['nflId'] == group['tacklerNflId'], 'x'] = row_22_x+0.05
    group.loc[group['nflId'] == group['tacklerNflId'], 'y'] = row_22_y-0.05
    return group
sorted_final2 = sorted_final2.groupby("uniqueid").apply(update_coordinates)
sorted_final2=sorted_final2.reset_index(level="uniqueid",drop=True)

In [ ]:
def convert_to_seconds(time_str):
    if pd.isna(time_str):  
        return None
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds
sorted_final2['gameClock'] = sorted_final2['gameClock'].apply(convert_to_seconds)
sorted_final2 = sorted_final2.groupby('uniqueid').apply(calculate_distance).reset_index(drop=True)
sorted_final3 = sorted_final2.sort_values(by=['uniqueid', 'side', 'distance_to_carrier'])

In [ ]:
'''
sorted_final3["offenseFormation"]=label_encoder.fit_transform(sorted_final2["offenseFormation"])
sorted_final3["side"]=label_encoder.fit_transform(sorted_final2["side"])
sorted_final3.drop(["displayName","nflId","club","playDirection","event","ballCarrierId","tackleOpportunity","possessionTeam","defensiveTeam"],axis=1,inplace=True)
sorted_final3['idx'] = sorted_final3.groupby(['uniqueid']).cumcount() + 1
sorted_final2.drop(["tacklerNflId"],axis=1,inplace=True)
sorted_final3 = sorted_final3.groupby('uniqueid').apply(calculate_distance).reset_index(drop=True)
sorted_final2['side'] = pd.Categorical(sorted_final2['side'], categories=["Offense", "Defense", "Football"], ordered=True)
sorted_final2 = sorted_final2.sort_values(by=['uniqueid', 'side', 'distance_to_carrier'])
sorted_final2.drop(["side","iscarrier"],axis=1,inplace=True)
sorted_final3.drop(["side","iscarrier"],axis=1,inplace=True)
'''
pivot_df2 = sorted_final3.pivot(index=['uniqueid','absoluteYardlineNumber','AbsolutePostplay','frameId',
                                   'quarter','down','yardsToGo','offenseFormation',"expectedPoints","tacklerNflId",
                                   'offenseScore','defenseScore','defendersInTheBox',
                                   'passProbability','offenseWinProbability','defenseWinProbability','gameClock'], columns='idx')
pivot_df2.columns = [f'{col[0]}{col[1]}' for col in pivot_df2.columns]
pivot_df2.reset_index(inplace=True)


In [ ]:
X_predict=pivot_df2.drop(["AbsolutePostplay","frameId","defenseWinProbability","uniqueid"],axis=1)


In [ ]:
pd.set_option("display.max_columns", None)
print(train3.columns.tolist())

In [ ]:
#pivot_df2.drop(["dir23","o23"],axis=1, inplace=True)
#y_true=pivot_df2["AbsolutePostplay"].values
X_predict=pivot_df2.drop(["AbsolutePostplay","frameId","defenseWinProbability","uniqueid","tacklerNflId"],axis=1).values
predTD = np.array([clf11.predict_proba(X_predict), clf21.predict_proba(X_predict), clf31.predict_proba(X_predict),
                       clf41.predict_proba(X_predict), clf51.predict_proba(X_predict),clf61.predict_proba(X_predict),
                       clf71.predict_proba(X_predict), clf81.predict_proba(X_predict)])
weighted_predictions3 = np.tensordot(predTD, weights, axes=((0),(0)))
final_probabilities3 = weighted_predictions3 / weighted_predictions3.sum(axis=1, keepdims=True)

prob3=final_probabilities3[:,1]
predictions3=np.where(prob3>=0.25,1,0)
pivot_df2["TD"]=predictions3

y_predict=stack_model.predict(X_predict)
pivot_df2["withoutTackler"]=y_predict
pivot_df2["yardssaved"]=pivot_df2["withoutTackler"]-pivot_df2["AbsolutePostplay"]

In [ ]:
pivot_df3=pivot_df2[pivot_df2["TD"]==0]
predYards=pivot_df3.drop(["TD","AbsolutePostplay","uniqueid","frameId","defenseWinProbability","tacklerNflId","yardssaved","withoutTackler"],axis=1).values
res=stack_model.predict(predYards)
pivot_df3["withoutTackler"]=res
TDs=pivot_df2[pivot_df2["TD"]==1]
TDs["withoutTackler"]=110
FINAL=pd.concat([TDs,pivot_df3])
FINAL["yardssaved"]=FINAL["withoutTackler"]-FINAL["AbsolutePostplay"]

In [ ]:
plays[plays["uniqueid"]==6573818175853]

In [ ]:
test.head(10)

In [ ]:
test["AbsolutePostplay"]=test["absoluteYardlineNumber"]+test["prePenaltyPlayResult"]
final_result=pd.merge(test,FINAL[["uniqueid","withoutTackler","yardssaved"]],on="uniqueid",how="left")
final_result.loc[final_result['tackleMade'] == 0, ["yardssaved"]] = np.nan
final_result.loc[final_result['tackleMade'] == 0, ["withoutTackler"]] = np.nan
final_result["potentialSaved"]=final_result["AbsolutePostplay"]-final_result["x1"]-3
final_result.loc[final_result['tackleMade'] == 1, ["potentialSaved"]] = np.nan

In [ ]:
final_result['1Dsaved'] = np.where(
    (final_result['withoutTackler'] - final_result['absoluteYardlineNumber'] > final_result['yardsToGo']) &
    (final_result['AbsolutePostplay'] - final_result['absoluteYardlineNumber'] < final_result['yardsToGo']),
    1, 
    0  
)
final_result['TDsaved'] = np.where(
    (final_result['withoutTackler'] >=110) &
    (final_result['AbsolutePostplay']<110),
    1, 
    0  
)
final_result['potential1Dsaved'] = np.where(
    (final_result['AbsolutePostplay'] -final_result["potentialSaved"]- final_result['absoluteYardlineNumber'] < final_result['yardsToGo']) &
    (final_result['AbsolutePostplay'] - final_result['absoluteYardlineNumber'] > final_result['yardsToGo']),
    1,  
    0  
)
final_result['potentialTDsaved'] = np.where(
    (final_result['AbsolutePostplay']>=110) &
    (final_result['AbsolutePostplay'] - final_result['potentialSaved'] < 110),
    1, 
    0   
)
final_result.loc[final_result['tackleMade'] == 0, ["1Dsaved","TDsaved"]] = np.nan
final_result.loc[final_result['tackleMade'] == 1, ["potential1Dsaved","potentialTDsaved"]] = np.nan
final_result["tacklerNflId"]=test_tackler
final_result = final_result.rename(columns={'tacklerNflId': 'nflId'})
final_res=pd.merge(final_result, players[["nflId","displayName"]], on="nflId",how="left")
result=final_res[["tackleMade","tackleProbability","displayName","withoutTackler","potentialSaved","yardssaved","1Dsaved","TDsaved","potential1Dsaved","potentialTDsaved"]]

In [ ]:
def calculate_impact_score(row):
    if row['tackleMade'] == 1:
        if row['TDsaved'] == 1:
            score = (row['yardssaved'] * 0.2 + 7)
        else:
            score = (row['yardssaved'] * 0.2 +
                     row['1Dsaved'] * 3)
    else:
        if row['potentialTDsaved'] == 1:
            score = (-row['potentialSaved'] * 0.2 - 7)
        else:
            score = (-row['potentialSaved'] * 0.2 -
                     row['potential1Dsaved'] * 3)
    return score
result['impactScore'] = result.apply(calculate_impact_score, axis=1)

def calculate_points_added(row):
    if row['impactScore'] > 0:
        return row['impactScore'] * (1 + (1 - row['tackleProbability']))
    else:
        return row['impactScore'] * row['tackleProbability']
result['SITR'] = result.apply(calculate_points_added, axis=1)

In [ ]:
grouped = result.groupby('displayName')['SITR']
average_points_added = grouped.mean()
total_points_added = grouped.sum()
combined_stats = pd.DataFrame({
    'Average SITR': average_points_added,
    'Total SITR': total_points_added
}).reset_index()
combined_stats["tackleOpportunities"]=combined_stats["Total SITR"]/combined_stats["Average SITR"]

In [ ]:
combined2=combined_stats[combined_stats["tackleOpportunities"]>7]
combined3=pd.merge(combined2, players, on="displayName",how="left")
combined3.drop(["nflId","height","weight","birthDate","collegeName"],axis=1,inplace=True)
combined3.loc[combined3['position'].isin(['QB','OLB','ILB','MLB']), 'position'] = 'LB'
combined3.loc[combined3['position'].isin(['G', 'RB','CB','FS','SS','DB']), 'position'] = 'DB'
combined3.loc[combined3['position'].isin(['DT','DE','NT']),'position']='DL'

In [ ]:
combined3[combined3["position"]=="DL"].sort_values(by="Average SITR",ascending=False)

In [ ]:
dl_df
dl_df.to_csv("dl_df.csv", index=False)  # index=False avoids extra row numbers

In [ ]:
lb_df
lb_df.to_csv("lb_df.csv", index=False)  # index=False avoids extra row numbers

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.patheffects as pe

# -------- data --------
df = lb_df.copy()

x = df["tackleOpportunities"].astype(float).to_numpy()
y = df["Average SITR"].astype(float).to_numpy()
tot = df["Total SITR"].astype(float).to_numpy()
names = df["displayName"].to_numpy()

# bubble size (robust scaling)
smin, smax = np.percentile(tot, [5, 95])
sizes = 50 + 1500 * (np.clip(tot, smin, smax) - smin) / (smax - smin + 1e-9)

# color by Average SITR
cmap = plt.get_cmap("turbo") if "turbo" in plt.colormaps() else plt.get_cmap("plasma")
norm = colors.Normalize(vmin=y.min(), vmax=y.max())

# -------- plot --------
fig = plt.figure(figsize=(11, 8))
ax = plt.gca()

sc = ax.scatter(
    x, y,
    s=sizes,
    c=norm(y), cmap=cmap,
    alpha=0.85, edgecolor="k", linewidths=0.3
)

# quadrant lines
x_med, y_med = np.median(x), np.median(y)
ax.axvline(x_med, linestyle="--", linewidth=1, alpha=0.5)
ax.axhline(y_med, linestyle="--", linewidth=1, alpha=0.5)

# -------- label top-4 by Total SITR --------
top4_idx = np.argsort(-tot)[:4]
xr = max(x.max() - x.min(), 1e-9)
yr = max(y.max() - y.min(), 1e-9)
dx0, dy0 = 0.06 * xr, 0.06 * yr

# force NW arrows for these two if they are in the top-4
special_nw = {"Bobby Okereke", "T.J. Edwards"}

for i in top4_idx:
    name = str(names[i])

    # label offset direction
    if name in special_nw:
        dx, dy = -abs(dx0)-3, abs(dy0)-0.2   # NW
    else:
        dx = dx0 if x[i] >= x_med else -dx0
        dy = dy0 if y[i] >= y_med else -dy0

    # arrow start position
    if name == "Pete Werner":
        # compute bubble radius in pixels from scatter size (points^2)
        r_pts = np.sqrt(sizes[i] / np.pi)              # points
        r_px  = r_pts * (fig.dpi / 72.0)               # pixels
        pad_px = 3.0                                   # tiny gap above the circle

        # transform (x,y) -> display px, add vertical offset, transform back to data
        p_disp = ax.transData.transform((x[i], y[i]-0.15))
        p_top_disp = (p_disp[0], p_disp[1] + r_px + pad_px)
        x_start, y_start = ax.transData.inverted().transform(p_top_disp)
        start_xy = (x_start, y_start)
    else:
        start_xy = (x[i], y[i])

    ann = ax.annotate(
        name,
        xy=start_xy,                           # arrow tail (starts at top of Pete's bubble)
        xytext=(x[i] + dx, y[i] + dy),         # label position
        textcoords="data",
        fontsize=9,
        ha="left", va="bottom",
        arrowprops=dict(arrowstyle="-", lw=0.9, alpha=0.9)
    )
    ann.set_path_effects([pe.withStroke(linewidth=2.5, foreground="white")])

# -------- cosmetics --------
cb = plt.colorbar(sc, ax=ax, pad=0.015)
cb.set_label("Average SITR")
cb.remove()

ax.set_xlabel("Tackle Opportunities",fontsize=18)
ax.set_ylabel("Average Solo Tackle Impact Rating",fontsize=18)
ax.set_title("Linebackers Tackle Efficiency and Volume", fontsize=18, pad=10)
ax.grid(True, alpha=0.2)

plt.tight_layout()
plt.savefig("lb_efficiency_volume.png", dpi=300, bbox_inches="tight")
plt.show()

# Optional save:
#plt.savefig("lb_efficiency_volume.png", dpi=300, bbox_inches="tight")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.patheffects as pe

# -------- data --------
df = db_df.copy()

x = df["tackleOpportunities"].astype(float).to_numpy()
y = df["Average SITR"].astype(float).to_numpy()
tot = df["Total SITR"].astype(float).to_numpy()
names = df["displayName"].to_numpy()

# bubble size (robust scaling)
smin, smax = np.percentile(tot, [5, 95])
sizes = 50 + 1500 * (np.clip(tot, smin, smax) - smin) / (smax - smin + 1e-9)

# color by Average SITR
cmap = plt.get_cmap("turbo") if "turbo" in plt.colormaps() else plt.get_cmap("plasma")
norm = colors.Normalize(vmin=y.min(), vmax=y.max())

# -------- plot --------
fig = plt.figure(figsize=(11, 8))
ax = plt.gca()

sc = ax.scatter(
    x, y,
    s=sizes,
    c=norm(y), cmap=cmap,
    alpha=0.85, edgecolor="k", linewidths=0.3
)

# quadrant lines
x_med, y_med = np.median(x), np.median(y)
ax.axvline(x_med, linestyle="--", linewidth=1, alpha=0.5)
ax.axhline(y_med, linestyle="--", linewidth=1, alpha=0.5)

# -------- label top-4 by Total SITR --------
# -------- label top-4 by Total SITR --------
top4_idx = np.argsort(-tot)[:4]
xr = max(x.max() - x.min(), 1e-9)
yr = max(y.max() - y.min(), 1e-9)
dx0, dy0 = 0.06 * xr, 0.06 * yr

for i in top4_idx:
    name = str(names[i])

    # vertical offset stays based on median
    dy = dy0 if y[i] >= y_med else -dy0

    # force horizontal direction for specific players
    if name == "Anthony Brown":
        dx = -abs(dx0)-2  # go left
        dy = abs(dy0)-0.5
    elif name == "Nate Hobbs":
        dx =  abs(dx0)  # go right
        dy = abs(dy0) - 0.5
    else:
        dx = dx0 if x[i] >= x_med else -dx0  # default logic

    ann = ax.annotate(
        name,
        xy=(x[i], y[i]),
        xytext=(x[i] + dx, y[i] + dy),
        textcoords="data",
        fontsize=9,
        ha="left", va="bottom",
        arrowprops=dict(arrowstyle="-", lw=0.9, alpha=0.9)
    )
    ann.set_path_effects([pe.withStroke(linewidth=2.5, foreground="white")])

# -------- cosmetics --------
# no colorbar (per your “slide bar” ban)
ax.set_xlabel("Tackle Opportunities", fontsize=18)
ax.set_ylabel("Average Solo Tackle Impact Rating", fontsize=18)
ax.set_title("Defensive Backs Tackle Efficiency and Volume", fontsize=18, pad=10)
ax.grid(True, alpha=0.2)

plt.tight_layout()
plt.savefig("db_efficiency_volume.png", dpi=300, bbox_inches="tight")
plt.show()

# Optional save:
#plt.savefig("db_bubble_top4.png", dpi=300, bbox_inches="tight")


In [ ]:
combined3[combined3["position"]=="LB"].sort_values(by="Average SITR",ascending=False)

In [ ]:
db_df
db_df.to_csv("db_df.csv", index=False)  # index=False avoids extra row numbers

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

# Work on a copy
df = lb_df.copy()
vals = df["Average SITR"].astype(float).dropna().to_numpy()

# Figure
fig = plt.figure(figsize=(11, 7))
ax = plt.gca()

# Choose bins
bins = 24

# Colormap for pretty bars
cmap = plt.get_cmap("turbo") if "turbo" in plt.colormaps() else plt.get_cmap("plasma")
norm = colors.Normalize(vmin=vals.min(), vmax=vals.max())

# Histogram
n, b, patches = ax.hist(
    vals,
    bins=bins,
    rwidth=0.92,
    edgecolor="white",
    linewidth=0.8,
    alpha=0.95
)

# Color each bar by its bin center
for i, p in enumerate(patches):
    center = 0.5 * (b[i] + b[i+1])
    p.set_facecolor(cmap(norm(center)))

# Colorbar keyed to Average SITR values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cb = plt.colorbar(sm, ax=ax, pad=0.015)
cb.set_label("Average SITR (bin color)")
cb.remove()
# Mean and median markers
mu = float(np.mean(vals))
med = float(np.median(vals))
ax.axvline(mu, linestyle="--", linewidth=1.5, label=f"Mean = {mu:.2f}")
ax.axvline(med, linestyle="-.", linewidth=1.5, label=f"Median = {med:.2f}")

# Labels/title
ax.set_xlabel("Average SITR")
ax.set_ylabel("Number of Players")
ax.set_title("Distribution of SITR for Linebackers")

# Light grid + tidy spines
ax.grid(True, alpha=0.18)
for spine in ax.spines.values():
    spine.set_alpha(0.5)

ax.legend(frameon=False)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

# Work on a copy
df = db_df.copy()
vals = df["Average SITR"].astype(float).dropna().to_numpy()

# Figure
fig = plt.figure(figsize=(11, 7))
ax = plt.gca()

# Choose bins
bins = 24

# Colormap for pretty bars
cmap = plt.get_cmap("turbo") if "turbo" in plt.colormaps() else plt.get_cmap("plasma")
norm = colors.Normalize(vmin=vals.min(), vmax=vals.max())

# Histogram
n, b, patches = ax.hist(
    vals,
    bins=bins,
    rwidth=0.92,
    edgecolor="white",
    linewidth=0.8,
    alpha=0.95
)

# Color each bar by its bin center
for i, p in enumerate(patches):
    center = 0.5 * (b[i] + b[i+1])
    p.set_facecolor(cmap(norm(center)))

# Colorbar keyed to Average SITR values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cb = plt.colorbar(sm, ax=ax, pad=0.015)
cb.set_label("Average SITR (bin color)")

# Mean and median markers
mu = float(np.mean(vals))
med = float(np.median(vals))
ax.axvline(mu, linestyle="--", linewidth=1.5, label=f"Mean = {mu:.2f}")
ax.axvline(med, linestyle="-.", linewidth=1.5, label=f"Median = {med:.2f}")

# Labels/title
ax.set_xlabel("Average SITR")
ax.set_ylabel("Number of Players")
ax.set_title("Distribution of SITR for Defensive Backs")

# Light grid + tidy spines
ax.grid(True, alpha=0.18)
for spine in ax.spines.values():
    spine.set_alpha(0.5)

ax.legend(frameon=False)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import FuncFormatter

# --- values ---
db_vals = db_df["Average SITR"].astype(float).to_numpy()
lb_vals = lb_df["Average SITR"].astype(float).to_numpy()

# --- shared bins ---
bins = 24
xmin = min(db_vals.min(), lb_vals.min())
xmax = max(db_vals.max(), lb_vals.max())
edges = np.linspace(xmin, xmax, bins + 1)
centers = 0.5 * (edges[:-1] + edges[1:])
bar_h = np.diff(edges)

# compare shapes; set to False for counts
density = True
h_db, _ = np.histogram(db_vals, bins=edges, density=density)
h_lb, _ = np.histogram(lb_vals, bins=edges, density=density)

# --- single colormap used for BOTH sides (same pattern) ---
cmap = plt.get_cmap("turbo") if "turbo" in plt.colormaps() else plt.get_cmap("plasma")
norm = colors.Normalize(vmin=centers.min(), vmax=centers.max())
colors_per_bin = [cmap(norm(c)) for c in centers]

fig = plt.figure(figsize=(12, 7))
ax = plt.gca()

# Bars: DB to the right, LB mirrored to the left
bars_db = ax.barh(centers,  h_db, height=bar_h, align="center",
                  edgecolor="white", linewidth=0.9, alpha=0.95, zorder=3)
bars_lb = ax.barh(centers, -h_lb, height=bar_h, align="center",
                  edgecolor="white", linewidth=0.9, alpha=0.95, zorder=3)

# Apply identical color pattern on both sides
for rect, col in zip(bars_db, colors_per_bin):
    rect.set_facecolor(col)
for rect, col in zip(bars_lb, colors_per_bin):
    rect.set_facecolor(col)

# Symmetric x-axis
m = max(h_db.max(), h_lb.max())
ax.set_xlim(-m * 1.2, m * 1.2)
ax.set_ylim(xmin, xmax)

# Bold center divider
ax.axvline(0, color="black", linewidth=5, alpha=0.95, zorder=5)

# Colored medians (red = DB, blue = LB)
med_db = float(np.median(db_vals))
med_lb = float(np.median(lb_vals))
ax.axhline(med_db, linestyle="--", linewidth=3.2, color="red",  label=f"DB median {med_db:.2f}",  zorder=4)
ax.axhline(med_lb, linestyle="-.", linewidth=3.2, color="blue", label=f"LB median {med_lb:.2f}", zorder=4)

# Huge side labels centered in each half (behind bars so they don't cover data)
ax.text(0.25, 0.50, "LB", color="blue", fontsize=48, fontweight="bold",
        ha="center", va="center", transform=ax.transAxes, alpha=0.16, zorder=1)
ax.text(0.75, 0.50, "DB", color="red",  fontsize=48, fontweight="bold",
        ha="center", va="center", transform=ax.transAxes, alpha=0.16, zorder=1)

# Bigger title/labels/ticks
ax.set_title("Solo Tackle Impact Rating Distribution— Linebackers (left) vs Defensive Backs (right)", fontsize=20, pad=12)
ax.set_xlabel("Density" if density else "Players", fontsize=18)
ax.set_ylabel("Average SITR", fontsize=18)
ax.tick_params(axis="both", labelsize=13)

# Pretty x-ticks: show absolute values
ax.xaxis.set_major_formatter(FuncFormatter(
    lambda v, pos: f"{abs(v):.2f}" if density else f"{int(abs(v))}"
))

ax.grid(True, axis="x", alpha=0.25)
ax.legend(frameon=False, loc="upper right", fontsize=12)

plt.tight_layout()
plt.savefig("stir_db_vs_lb.png", dpi=300, bbox_inches="tight")  # high-res PNG
plt.show()


In [ ]:
combined3[combined3["position"]=="DB"].sort_values(by="Average SITR",ascending=False)